### Start here

In [ ]:
# %run "0a_Workspace_setup.ipynb"
%run "0b_Create_poi_files.ipynb"

In [ ]:
poi_id_sel = None

#### Useful grouping functions for HRUs grouping--adopted from pyPRMS

In [ ]:
def subset_stream_network(dag_ds, uscutoff_seg, dsmost_seg):
    """Extract subset of stream network

    :param dag_ds: Directed, acyclic graph of downstream stream network
    :param uscutoff_seg: List of upstream cutoff segments
    :param dsmost_seg: List of outlet segments to start extraction from

    :returns: Stream network of extracted segments
    """

    # taken from Bandit bandit_helpers.py

    # Create the upstream graph
    dag_us = dag_ds.reverse()

    # Trim the u/s graph to remove segments above the u/s cutoff segments
    try:
        for xx in uscutoff_seg:
            try:
                dag_us.remove_nodes_from(nx.dfs_predecessors(dag_us, xx))

                # Also remove the cutoff segment itself
                dag_us.remove_node(xx)
            except KeyError:
                print(f"WARNING: nhm_segment {xx} does not exist in stream network")
    except TypeError:
        print(
            "\nSelected cutoffs should at least be an empty list instead of NoneType."
        )

    # =======================================
    # Given a d/s segment (dsmost_seg) create a subset of u/s segments

    # Get all unique segments u/s of the starting segment
    uniq_seg_us: Set[int] = set()
    if dsmost_seg:
        for xx in dsmost_seg:
            try:
                pred = nx.dfs_predecessors(dag_us, xx)
                uniq_seg_us = uniq_seg_us.union(
                    set(pred.keys()).union(set(pred.values()))
                )
            except KeyError:
                print(f"KeyError: Segment {xx} does not exist in stream network")

        # Get a subgraph in the dag_ds graph and return the edges
        dag_ds_subset = dag_ds.subgraph(uniq_seg_us).copy()

        node_outlets = [ee[0] for ee in dag_ds_subset.edges()]
        true_outlets = set(dsmost_seg).difference(set(node_outlets))

        # Add the downstream segments that exit the subgraph
        for xx in true_outlets:
            nhm_outlet = list(dag_ds.neighbors(xx))[0]
            dag_ds_subset.add_node(
                nhm_outlet, style="filled", fontcolor="white", fillcolor="grey"
            )
            dag_ds_subset.add_edge(xx, nhm_outlet)
            dag_ds_subset.nodes[xx]["style"] = "filled"
            dag_ds_subset.nodes[xx]["fontcolor"] = "white"
            dag_ds_subset.nodes[xx]["fillcolor"] = "blue"
    else:
        # No outlets specified so pull the full model
        dag_ds_subset = dag_ds

    return dag_ds_subset


def hrus_by_seg(pdb, segs):
    # segs: global segment IDs

    if isinstance(segs, int):
        segs = [segs]
    elif isinstance(segs, KeysView):
        segs = list(segs)

    seg_hrus = {}
    seg_to_hru = pdb.seg_to_hru

    # Generate stream network for the model
    dag_streamnet = pdb.stream_network()

    for cseg in segs:
        # Lookup segment for the current POI
        dsmost_seg = [cseg]

        # Get subset of stream network for given POI
        dag_ds_subset = subset_stream_network(dag_streamnet, set(), dsmost_seg)

        # Create list of segments in the subset
        toseg_idx = list(set(xx[0] for xx in dag_ds_subset.edges))

        # Build list of HRUs that contribute to the POI
        final_hru_list = []

        for xx in toseg_idx:
            try:
                for yy in seg_to_hru[xx]:
                    final_hru_list.append(yy)
            except KeyError:
                # print(f'Segment {xx} has no HRUs connected to it') # comment this out and add pass to not print the KeyError
                pass
        final_hru_list.sort()

        seg_hrus[cseg] = final_hru_list

    return seg_hrus


def hrus_by_poi(pdb, poi):
    if isinstance(poi, str):
        poi = [poi]
    elif isinstance(poi, KeysView):
        poi = list(poi)

    poi_hrus = {}
    nhm_seg = pdb.get("nhm_seg").data
    pois_dict = pdb.poi_to_seg
    seg_to_hru = pdb.seg_to_hru

    # Generate stream network for the model
    dag_streamnet = pdb.stream_network()

    for cpoi in poi:
        # Lookup global segment id for the current POI
        dsmost_seg = [nhm_seg[pois_dict[cpoi] - 1]]

        # Get subset of stream network for given POI
        dag_ds_subset = subset_stream_network(dag_streamnet, set(), dsmost_seg)

        # Create list of segments in the subset
        toseg_idx = list(set(xx[0] for xx in dag_ds_subset.edges))

        # Build list of HRUs that contribute to the POI
        final_hru_list = []

        for xx in toseg_idx:
            try:
                for yy in seg_to_hru[xx]:
                    final_hru_list.append(yy)
            except KeyError:
                # Not all segments have HRUs connected to them
                # print(f'{cpoi}: Segment {xx} has no HRUs connected to it')
                pass
        final_hru_list.sort()
        poi_hrus[cpoi] = final_hru_list

    return poi_hrus

### Open Model Output file and merge selected output with the geodatabase  
##### Open up the model output, this is a NetCDF file of all varables selected for output in the "Run_Model" notebook.

In [ ]:
from datetime import datetime

WY_start = "1979-10-01"
# plot_start_dt = datetime.strptime(WY_start, "%Y-%m-%d").date()


WY_end = "2021-09-30"
# plot_end_dt = datetime.strptime(WY_end, "%Y-%m-%d").date()

with xr.open_dataset(out_dir / "seg_outflow.nc") as model_output:
    output = model_output.sel(time=slice(WY_start, WY_end))
    year_list = list(set(((output.time.dt.year).values).ravel().tolist()))

    # output_var_start_date = output.time.values[0]
    output_var_start_date = (pd.to_datetime(str(output.time.values[0]))).strftime(
        "%Y-%m-%d"
    )
    plot_start_dt = datetime.strptime(output_var_start_date, "%Y-%m-%d").date()

    output_var_end_date = (pd.to_datetime(str(output.time.values[-1]))).strftime(
        "%Y-%m-%d"
    )
    plot_end_dt = datetime.strptime(output_var_end_date, "%Y-%m-%d").date()
    del output

output_var_list = set([vv.stem for vv in out_dir.glob("*.nc")])
# This vars dimensioned by nseg,in another notebook.
output_var_list = list(
    output_var_list - {"seg_outflow", "model_custom_output", "hru_streamflow_out"}
)
print(f"{output_var_list=}")

# Note: Model output is in calendar years, so must remove first year from the list to show available WY's in the data
year_list.remove(year_list[0])
year_list = [str(x) for x in year_list]
# output_var_list

In [ ]:
# Check output variable units
from ast import literal_eval

for var in output_var_list:
    # units = f'output.{var}.units'
    ds = xr.open_dataset(out_dir / f"{var}.nc")
    print(f"{var}: {ds[var].units}")
    ds.close()

# Select a spatial output variable (by HRU) to process and a year to viewlist above dimensioned by "nhru."

In [ ]:
from ipywidgets import widgets
from ipywidgets import Text, HBox, VBox, Box
from IPython.display import display

##### Varibale selection widget
output_var_sel = output_var_list[
    1
]  # Set a default value so that the notebook will run without selection

style_date = {"description_width": "initial"}

style_var = {"description_width": "initial"}

v = widgets.Dropdown(
    options=output_var_list,
    value=output_var_list[1],
    description="Output variable for map and plot:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global output_var_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        output_var_sel = v.value
        # sel_flag = True


v.observe(on_change)
# display(v)

##### Year selection widget
list_of_years = year_list.copy()
list_of_years.append(
    "mean_annual"
)  # Append the mean annual so that the default will not be a year
sel_year = list_of_years[
    -1
]  # Set a default value so that the notebook will run without selection

yr = widgets.Dropdown(
    options=list_of_years,
    value=list_of_years[-1],
    description="Time step (year) for map:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global sel_year  # Have to set the var as global so that it is carried outside of the fucntion to the notebook
    if change["type"] == "change" and change["name"] == "value":
        sel_year = yr.value


yr.observe(on_change)

#####################################
# Add widgets for start and end date, and for picking the gage here.

global plot_start_dt, plot_end_dt, output_var_end_date, output_var_start_date, plot_end_date, plot_start_date, poi_id_sel
plot_end_date = output_var_end_date
plot_start_date = output_var_start_date

st_date = widgets.DatePicker(
    description="Start date for plot:",
    disabled=False,
    value=plot_start_dt,
    min=plot_start_dt,
    max=plot_end_dt,
    layout=widgets.Layout(width="25%"),
    style=style_date,
)


def on_change_st_date(change):
    global plot_start_date  # , sel_flag
    if change["type"] == "change" and change["name"] == "value":
        plot_start_date = st_date.value.strftime("%Y-%m-%d")
        # sel_flag = True


st_date.observe(on_change_st_date)

#### End Date
end_date = widgets.DatePicker(
    description="End date for plot:",
    disabled=False,
    value=plot_end_dt,
    min=plot_start_dt,
    max=plot_end_dt,
    layout=widgets.Layout(width="25%"),
    style=style_date,
)


def on_change_end_date(change):
    global plot_end_date  # , sel_flag
    if change["type"] == "change" and change["name"] == "value":
        plot_end_date = end_date.value.strftime("%Y-%m-%d")
        # sel_flag = True


end_date.observe(on_change_end_date)

#################################################
v2 = widgets.Combobox(
    # value=poi_df.poi_id.tolist()[0],
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change2(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v2.value


v2.observe(on_change2)

display(VBox([v, yr, st_date, end_date, v2]))

### Click here and select "Run Selected Cell and All Below" from the "Run" menu in the toolbar.

In [ ]:
# JLM TODO
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

In [ ]:
con.print(f"{output_var_sel}, {sel_year}, {poi_id_sel}")

In [ ]:
# # Get data for map and first plot
# with xr.open_dataset(custom_output_file) as model_output:
#     output_var = model_output.sel(time=slice(WY_start, WY_end))
#     # output_var_daily = getattr(output_var, output_var_sel)
#     output_var_daily = output_var[output_var_sel]
#     output_var_annual = (
#         getattr(output_var, output_var_sel).resample(time="A-SEP").sum()
#     )  # Water year
#     output_var_monthly = (
#         getattr(output_var, output_var_sel).resample(time="m").sum()
#     )  # monthly

# ### actet_daily = modelobsdat.hru_actet.sel(time=slice(aet_start, aet_end))#Slice example

In [ ]:
# JLM TODO: why are these sums? below means are calculated...
with xr.load_dataarray(out_dir / f"{output_var_sel}.nc") as da:
    # these machinations are to keep downstream things as they were before some refactoring
    da = da.to_dataset().rename_dims({"nhm_id": "nhru"})[da.name]
    output_var_daily = da.sel(time=slice(WY_start, WY_end))
    output_var_monthly = output_var_daily.resample(time="m").sum()
    # Water year annual
    output_var_annual = output_var_daily.resample(time="A-SEP").sum()

In [ ]:
# xr.testing.assert_equal(output_var_daily, output_var_daily2)
# xr.testing.assert_equal(output_var_monthly, output_var_monthly2)
# xr.testing.assert_equal(output_var_annual, output_var_annual2)

#### Change data_array from .nc to a DataFrame
#### This dataframe will be used to 1) output a .csv, 2) map data to the GIS, and 3) plot data

In [ ]:
ds_output_var_annual = output_var_annual.copy()
df_output_var_annual = ds_output_var_annual.to_dataframe(dim_order=["time", "nhru"])
df_output_var_annual.reset_index(inplace=True, drop=False)
df_output_var_annual.set_index(
    "nhm_id", inplace=True, drop=True
)  # resets the index to that new value and type

df_output_var_annual["year"] = pd.DatetimeIndex(df_output_var_annual["time"]).year
df_output_var_annual.year = df_output_var_annual.year.astype(str)
df_output_var_annual.rename(
    columns={output_var_sel: "output_var"}, inplace=True
)  # Rename the column to a general heading for later code

df_output_var_annual.drop(columns=["time", "nhru"], inplace=True)

# year_list = list(df_output_var_annual['year'].unique())
# year_list
year_list_str = [str(x) for x in year_list]

# df_output_var_annual

#### Export a database of selected output var data as a table (.csv)

In [ ]:
table = pd.pivot_table(
    df_output_var_annual,
    values="output_var",
    index=["nhm_id"],
    columns=["year"],
).round(2)
table.reset_index(inplace=True, drop=False)
# table

In [ ]:
# # Select only hru's in a domain: Walla Walla subabasin model
# subbasin_hrus = [99000,99003,98963,99049,98957,99050,98947,98951,98874,98875,98753,98764,98974,99147,98960,98983,99899,
#                  99900,98802,98804,98789,98792,98813,98814,98803,98811,98801,98806,98939,98962,98936,98940,98937,98941,
#                  98903,98904,98861,98884,99581,99618,99030,99141,99043,99065,98930,98935,99042,99081,99920,99921,99182,
#                  99204,99183,99202,99162,99167,99125,99160,99113,99114,99075,99076,99132,99159,99131,99136,99580,99617,
#                  99145,99173,99170,99219,99574,99614,98905,98907,98901,98915,98876,98877,98828,98933,98934,99566,99615,
#                  99237,99261,99578,99616,99056,99099]
# table_sub = table.loc[table['nhm_id'].isin(subbasin_hrus),:]
# table_sub

In [ ]:
output_filename = f"{shapefile_dir}/{output_var_sel}_annual_{da.units}.csv"  # Writes gpd GeoDataFrame our t as a shapefile for fun
table.to_csv(output_filename)

#### Merge in the geometry from the geodatabase with the dataframe

In [ ]:
gdf_output_var_annual = hru_gdf.merge(table, on="nhm_id")
gdf_output_var_annual.info()

In [ ]:
gdf_output_var_annual = hru_gdf.merge(table, on="nhm_id")
gdf_output_var_annual.drop(
    columns=["hru_lat", "hru_lon", "hru_segment_nhm", "model_idx"], inplace=True
)

In [ ]:
# gdf_output_var_annual
# isinstance(gdf_output_var_annual, gpd.GeoDataFrame)

In [ ]:
# Make the mean annual value for the model simulation period, year_list
for nhm_id in gdf_output_var_annual:
    gdf_output_var_annual["mean_annual"] = gdf_output_var_annual[year_list].mean(axis=1)
# gdf_output_var_annual

In [ ]:
# gdf_output_var_annual

### Make an interactive map with model output

#### Determine the minimum and maximum values for annual recharge for the legend value bar.

In [ ]:
value_min = gdf_output_var_annual[year_list].min().min()
value_max = gdf_output_var_annual[year_list].max().max()

# value_mean = gdf_output_var_annual[[sel_years]].mean(axis=1)
# value_med = gdf_output_var_annual[[sel_years]].median(axis=1)

print(f"The minimum and maximum annual values are: {value_min}, {value_max}.")
print(
    "This will be used to set a values in the legend bar when mapping the annual data."
)

#### Create the basemap

In [ ]:
# Convert the gdf projection for mapping
hru_gdf_map = hru_gdf.to_crs(crs)

In [ ]:
# Set pfile lat, lon derived for starting point of folium plot windows. Zoom also set here.
lat = hru_gdf_map["hru_lat"].mean()
lon = hru_gdf_map["hru_lon"].mean() - 1
zoom = 7

In [ ]:
# Define basemaps for map
USGStopo_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_topo",
    zoom_start=zoom,
    name="USGSTopo",
)
USGSHydroCached_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSHydroCached/MapServer/tile/{z}/{y}/{x}",
    attr="USGSHydroCached",
    zoom_start=zoom,
    name="USGSHydroCached",
)

In [ ]:
# Create inset map object
minimap = plugins.MiniMap(
    tile_layer="OpenStreetMap",
    # attr = 'USGS_topo',
    position="topleft",
    # zoom_level_offset=- 4,
    height=200,
    width=200,
    collapsed_height=25,
    collapsed_width=25,
    zoom_level_fixed=5,
    toggle_display=True,
)

In [ ]:
# set style functions
style_function_hru_map = lambda x: {
    "opacity": 1,
    "fillColor": "#00000000",  #'goldenrod',
    "color": "tan",
    "weight": 1.5,
}
highlight_function_hru_map = lambda x: {
    "opacity": 0.5,
    "color": "gray",
    "fillColor": "gray",
    "weight": 3,
}
style_function_seg_map = lambda x: {"opacity": 1, "color": "#217de7", "weight": 2}
highlight_function_seg_map = lambda x: {"opacity": 1, "color": "white", "weight": 4}
transparent = lambda x: {
    "fillColor": "#00000000",
    "color": "#00000000",
    "weight": 4,
}
cp_style_function = lambda feature: {
    "fillColor": linear(var_sel_color_dict[feature["id"]]),
    "color": "tan",
    "weight": 1,
    # "dashArray": "5, 5",
    "fillOpacity": 0.3,
}

In [ ]:
# Set output bins for color dictionary
hru_gdf_copy = gdf_output_var_annual.copy().reset_index().to_crs(crs)
hru_gdf_copy["nhm_id"] = hru_gdf_copy["nhm_id"].astype(str)
hru_gdf_copy.set_index("nhm_id", inplace=True, drop=False)

var_subset_df = gdf_output_var_annual.loc[:, ["nhm_id", str(sel_year)]]
var_subset_df["nhm_id"] = var_subset_df["nhm_id"].astype(str)
var_subset_df.rename(columns={f"{sel_year}": "var_value"}, inplace=True)
var_subset_df["var_value"] = np.round(var_subset_df["var_value"], 4)
var_subset_df.set_index("nhm_id", inplace=True, drop=False)

value_min = np.round(var_subset_df["var_value"].min(), 8)
value_max = np.round(var_subset_df["var_value"].max(), 8)


var_sel_color_dict = pd.Series(
    var_subset_df.var_value.values, index=var_subset_df.nhm_id
).to_dict()

# Making par_bins
sdv = var_subset_df["var_value"].std()
mean = var_subset_df["var_value"].mean()

var_bins = [
    value_min,
    np.round(value_min + (0.25 * (mean - value_min)), 5),
    np.round(value_min + (0.50 * (mean - value_min)), 5),
    np.round(value_min + (0.75 * (mean - value_min)), 5),
    np.round(mean, 3),
    np.round(value_max - (0.75 * (value_max - mean)), 5),
    np.round(value_max - (0.50 * (value_max - mean)), 5),
    np.round(value_max - (0.25 * (value_max - mean)), 5),
    value_max,
]

### Zoom to Gage

In [ ]:
# # Zoom to gage, just uncomment the lines below

# poi_lookup = poi_id_sel
# lat = poi_df.loc[poi_df.poi_id==poi_lookup, 'latitude'].values[0]
# lon = poi_df.loc[poi_df.poi_id==poi_lookup, 'longitude'].values[0]
# zoom = 12

# if poi_id_sel:
#     poi_lookup = poi_id_sel
#     lat = poi_df.loc[poi_df.poi_id==poi_lookup, 'latitude'].values[0]
#     lon = poi_df.loc[poi_df.poi_id==poi_lookup, 'longitude'].values[0]
#     zoom = 12
# else:
#     lat = hru_gdf_map['hru_lat'].mean()
#     lon =  hru_gdf_map['hru_lon'].mean()-1
#     zoom = 7

### Create Map

In [ ]:
# Create Map
if poi_id_sel:
    poi_lookup = poi_id_sel
    lat = poi_df.loc[poi_df.poi_id == poi_lookup, "latitude"].values[0]
    lon = poi_df.loc[poi_df.poi_id == poi_lookup, "longitude"].values[0]
    zoom = 12
    # poi_id_sel= None
else:
    lat = hru_gdf_map["hru_lat"].mean()
    lon = hru_gdf_map["hru_lon"].mean() - 1
    zoom = 7

m3 = folium.Map(
    location=[lat, lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)
folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_Topo",
    zoom_start=zoom,
    name="USGS Topography",
    show=False,
).add_to(m3)

#########################################################################################
hru_gdf_copy = gdf_output_var_annual.copy().reset_index().to_crs(crs)
hru_gdf_copy["nhm_id"] = hru_gdf_copy["nhm_id"].astype(str)
hru_gdf_copy.set_index("nhm_id", inplace=True, drop=False)

var_subset_df = gdf_output_var_annual.loc[:, ["nhm_id", str(sel_year)]]
var_subset_df["nhm_id"] = var_subset_df["nhm_id"].astype(str)
var_subset_df.rename(columns={f"{sel_year}": "var_value"}, inplace=True)
var_subset_df["var_value"] = np.round(var_subset_df["var_value"], 4)
var_subset_df.set_index("nhm_id", inplace=True, drop=False)

value_min = np.round(var_subset_df["var_value"].min(), 8)
value_max = np.round(var_subset_df["var_value"].max(), 8)


var_sel_color_dict = pd.Series(
    var_subset_df.var_value.values, index=var_subset_df.nhm_id
).to_dict()

# Making par_bins
sdv = var_subset_df["var_value"].std()
mean = var_subset_df["var_value"].mean()

var_bins = [
    value_min,
    np.round(value_min + (0.25 * (mean - value_min)), 5),
    np.round(value_min + (0.50 * (mean - value_min)), 5),
    np.round(value_min + (0.75 * (mean - value_min)), 5),
    np.round(mean, 3),
    np.round(value_max - (0.75 * (value_max - mean)), 5),
    np.round(value_max - (0.50 * (value_max - mean)), 5),
    np.round(value_max - (0.25 * (value_max - mean)), 5),
    value_max,
]

#################################################
fig, ax = plt.subplots(figsize=(18, 0.5))
fig.subplots_adjust(bottom=0.5)

cmap = mplib.colors.ListedColormap(
    [
        "#8B0000",
        "#AC4800",
        "#CD9100",
        "#EEDA00",
        "#DADA13",
        "#91913B",
        "#484863",
        "#00008B",
    ]
)
cmap.set_over("0.25")
cmap.set_under("0.75")

bounds = var_bins
norm = mplib.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mplib.colorbar.ColorbarBase(
    ax,
    cmap=cmap,
    norm=norm,
    boundaries=[0] + bounds + [13],
    extend=None,
    ticks=bounds,
    spacing="uniform",
    orientation="horizontal",
    alpha=0.45,
)
cb2.set_label(
    f"Discrete {output_var_sel} intervals"
)  # , {pdb.get(output_var_sel).units}')

fig.set_facecolor("lightgray")
fig.show()

#######################################################

linear = cm.StepColormap(
    colors=[
        "#8B0000",
        "#AC4800",
        "#CD9100",
        "#EEDA00",
        "#DADA13",
        "#91913B",
        "#484863",
        "#00008B",
    ],
    index=var_bins,
    vmin=0.00,
    vmax=0.05,
    caption="Total Standard deviation at the point[mm]",
    # tick_labels= ('0.01', '0.02', '0.03', '0.04')
)
popup_hru = folium.GeoJsonPopup(
    fields=["nhm_id", str(sel_year)],
    aliases=["HRU", f"{output_var_sel} for {sel_year}"],
    labels=True,
    localize=True,
    style=(
        "font-size: 16px;"
    ),  # Note that this tooltip style sets the style for all tool_tips.
    # background-color: #F0EFEF;border: 2px solid black;font-family: arial; padding: 10px; background-color: #F0EFEF;
)

hru_map = folium.GeoJson(
    hru_gdf_copy,
    style_function=cp_style_function,  # style_function_hru_map,
    highlight_function=highlight_function_hru_map,
    name="NHM HRUs",
    popup=popup_hru,
    z_index_offset=40002,
).add_to(m3)

# tooltip_hru=folium.GeoJsonPopup(fields= ["nhm_id",str(sel_year)],
#                                   aliases=["HRU", "var value"],
#                                   labels=True)

# tooltip_hru=folium.GeoJsonTooltip(fields= ["nhm_id",str(sel_year)],
#                                   aliases=["HRU", f"{output_var_sel} for {sel_year}"],
#                                   labels=True,
#                                   localize=True,
#                                   style=("background-color: #F0EFEF;border: 2px solid black;font-family: arial; font-size: 16px; padding: 10px;"),
#                                      )


# Add tool tip to map
# hru_map.add_child(tooltip_hru)


# Create and add segments map
popup_seg = folium.GeoJsonPopup(
    fields=["nhm_seg", "tosegment_nhm"],
    aliases=["segment", "flows to segment"],
    labels=True,
    localize=False,
)
seg_map = folium.GeoJson(
    seg_gdf,
    style_function=style_function_seg_map,
    highlight_function=highlight_function_seg_map,  # lambda feature: {"fillcolor": "white", "color": "white"},
    name="NHM Segments",
    control=True,
    popup=popup_seg,
    z_index_offset=40003,
).add_to(m3)

# tooltip_seg=folium.GeoJsonTooltip(fields= ["nhm_seg"],
#                                   aliases=["Segment"],
#                                   labels=True)
# seg_map.add_child(tooltip_seg)


# add POI markers

marker_cluster = MarkerCluster(
    name="All the POIs",
    overlay=True,
    control=True,
    icon_create_function=None,
    disableClusteringAtZoom=9,
    z_index_offset=5000,
)

marker_cluster_label_poi = MarkerCluster(
    name="All the POI labels",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=9,
    z_index_offset=4004,
)

marker_cluster_label_hru = MarkerCluster(
    name="All HRU labels",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=9,
    z_index_offset=4005,
)


m3.add_child(minimap)

# Create Popup for the byHRU_map
# popup_byHRU=folium.GeoJsonPopup(fields= ['nhm_id_label', str(sel_year)],
#                                 aliases=['HRU' , output_var_sel],
#                                 labels=True, localize = True)

label_coord_x = 20
label_coor_y = 10

for idx, row in poi_df.iterrows():
    poi_id = row["poi_id"]
    marker = folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        name=row["poi_id"],
        popup=folium.Popup(
            f'<font size="3px">{row["poi_id"]}<br>{row["poi_name"]}</font>',
            max_width=280,
            max_height=2000,
        ),
        radius=4,
        weight=2,
        color="black",
        fill=True,
        fill_color="Yellow",
        fill_opacity=1.0,
        draggable=True,
        z_index_offset=4006,
    ).add_to(marker_cluster)

    # marker_cluster.add_child(marker)
    text = f'{row["poi_id"]}'
    label_lat = row["latitude"] - 0.01
    label_lon = row["longitude"]

    marker_label = folium.map.Marker(
        [label_lat, label_lon],
        z_index_offset=4007,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12pt; font-weight: bold">%s</div>' % text,
        ),
    ).add_to(marker_cluster_label_poi)


hru_gdf_label = hru_gdf.copy()
hru_gdf_label.reset_index(inplace=True, drop=False)

for idx, row in hru_gdf_label.iterrows():
    text = f'{row["nhm_id"]}'
    label_lat = row["hru_lat"]
    label_lon = row["hru_lon"]
    marker_label = folium.map.Marker(
        [label_lat, label_lon],
        z_index_offset=4008,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html='<div style="font-family: verdona; font-size: 10pt; font-weight: bold; color: black; text-shadow: 1px 1px 2px white;">%s</div>'
            % text,
        ),
    ).add_to(marker_cluster_label_hru)

marker_cluster.add_to(m3)
marker_cluster_label_poi.add_to(m3)
marker_cluster_label_hru.add_to(m3)

plugins.Fullscreen(position="topleft").add_to(m3)


folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m3)

con.print(f"")
con.print(f"")
con.print(f"")
con.print(f"NHM simulated {sel_year} {output_var_sel}", style="u bold black")

m3

### Compare HRU outputs by catchment for selected year

#### Make groups of HRUs for POIs

In [ ]:
# Group HRUs for each gage--these are all HRUs that contribute to each gage.

hru_gdf.reset_index(inplace=True)

# Create poi list from the POI dataframe
poi_list = poi["poi_gage_id"].values.tolist()

# Make a dictionary of pois and the list of HRUs in the contributing area for each poi
hru_poi_dict = hrus_by_poi(pdb, poi_list)  # Helper function from pyPRMS

# Sort the dictionary: this is important for the reverse dictionary (next step)
# to accurately give a poi_group to hrus that contribute to a downstream-gage
sorted_items = sorted(
    hru_poi_dict.items(), key=lambda item: -len(item[1])
)  # the - reverses the sorting order
hru_poi_dict = dict(sorted_items[:])

reversed_hru_poi_dict = {val: key for key in hru_poi_dict for val in hru_poi_dict[key]}

# assigns poi_group value to all hrus#Keep for later application
hru_gdf["poi_group"] = hru_gdf["nhm_id"].map(reversed_hru_poi_dict)

In [ ]:
# Create a dataframe of ANNUAL recharge values for each HRU
hru_area_df = hru_gdf[["hru_area", "nhm_id"]].set_index(
    "nhm_id", drop=True
)  # Consider a dictionary from the par file and using .map() instead of merge

ds_output_var_annual_basin = output_var_annual.copy().sel(
    time=slice(plot_start_date, plot_end_date)
)

df_output_var_annual_basin = ds_output_var_annual_basin.to_dataframe(
    dim_order=["time", "nhru"]
)
df_output_var_annual_basin.reset_index(inplace=True, drop=False)

df_output_var_annual_basin = df_output_var_annual_basin.merge(
    hru_area_df, how="left", right_index=True, left_on="nhm_id"
)

# # add the HRU area to the dataframe
# for idx, row in df_output_var_annual_basin.iterrows():
#     df_output_var_annual_basin.loc[idx, 'hru_area'] = hru_gdf.loc[hru_gdf.nhm_id == row.nhm_id, 'hru_area'].item()

# Add recharge volume to the dataframe
df_output_var_annual_basin["vol"] = (
    df_output_var_annual_basin[output_var_sel] * df_output_var_annual_basin["hru_area"]
)

# Drop unneeded columns
df_output_var_annual_basin.drop(columns=["nhru"], inplace=True)
# df_output_var_annual_basin

In [ ]:
# Create a dataframe of MONTHLY recharge values for each HRU
ds_output_var_monthly_basin = output_var_monthly.copy().sel(
    time=slice(plot_start_date, plot_end_date)
)

df_output_var_monthly_basin = ds_output_var_monthly_basin.to_dataframe(
    dim_order=["time", "nhru"]
)
df_output_var_monthly_basin.reset_index(inplace=True, drop=False)

# add the HRU area to the dataframe
df_output_var_monthly_basin = df_output_var_monthly_basin.merge(
    hru_area_df, how="left", right_index=True, left_on="nhm_id"
)

# Add recharge volume to the dataframe
df_output_var_monthly_basin["vol"] = (
    df_output_var_monthly_basin[output_var_sel]
    * df_output_var_monthly_basin["hru_area"]
)

# Drop unneeded columns
df_output_var_monthly_basin.drop(columns=["nhru"], inplace=True)

In [ ]:
# Create a dataframe of Daily recharge values for each HRU
ds_output_var_daily_basin = output_var_daily.copy().sel(
    time=slice(plot_start_date, plot_end_date)
)

df_output_var_daily_basin = ds_output_var_daily_basin.to_dataframe(
    dim_order=["time", "nhru"]
)
df_output_var_daily_basin.reset_index(inplace=True, drop=False)

# add the HRU area to the dataframe
df_output_var_daily_basin = df_output_var_daily_basin.merge(
    hru_area_df, how="left", right_index=True, left_on="nhm_id"
)

# Add recharge volume to the dataframe
df_output_var_daily_basin["vol"] = (
    df_output_var_daily_basin[output_var_sel] * df_output_var_daily_basin["hru_area"]
)

# Drop unneeded columns
df_output_var_daily_basin.drop(columns=["nhru"], inplace=True)

## Paste the poi_id in the field below

In [ ]:
v3 = widgets.Combobox(
    value=poi_id_sel,
    placeholder="Select Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
)


def on_change3(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v3.value


v3.observe(on_change3)

display(v3)

#### Click here and select "Run Selected Cell and All Below" from the "Run" menu in the toolbar.

In [ ]:
# HRU color list
import random

plot_colors = [
    "aliceblue",
    "aqua",
    "aquamarine",
    "azure",
    "beige",
    "bisque",
    "black",
    "blue",
    "blueviolet",
    "brown",
    "burlywood",
    "cadetblue",
    "chartreuse",
    "chocolate",
    "coral",
    "cornflowerblue",
    "crimson",
    "cyan",
    "darkblue",
    "darkcyan",
    "darkgoldenrod",
    "darkgray",
    "darkgrey",
    "darkgreen",
    "darkkhaki",
    "darkmagenta",
    "darkolivegreen",
    "darkorange",
    "darkorchid",
    "darkred",
    "darksalmon",
    "darkseagreen",
    "darkslateblue",
    "darkslategray",
    "darkslategrey",
    "darkturquoise",
    "darkviolet",
    "deeppink",
    "deepskyblue",
    "dodgerblue",
    "firebrick",
    "forestgreen",
    "fuchsia",
    "gainsboro",
    "goldenrod",
    "green",
    "greenyellow",
    "honeydew",
    "hotpink",
    "indianred",
    "indigo",
    "lavender",
    "lawngreen",
    "lime",
    "limegreen",
    "magenta",
    "maroon",
    "mediumaquamarine",
    "mediumblue",
    "mediumorchid",
    "mediumpurple",
    "mediumseagreen",
    "mediumslateblue",
    "mediumspringgreen",
    "mediumturquoise",
    "mediumvioletred",
    "midnightblue",
    "mintcream",
    "moccasin",
    "navy",
    "olive",
    "olivedrab",
    "orange",
    "orangered",
    "orchid",
    "palegreen",
    "paleturquoise",
    "palevioletred",
    "papayawhip",
    "peachpuff",
    "peru",
    "pink",
    "plum",
    "powderblue",
    "purple",
    "red",
    "rosybrown",
    "royalblue",
    "rebeccapurple",
    "saddlebrown",
    "salmon",
    "sandybrown",
    "seagreen",
    "sienna",
    "silver",
    "skyblue",
    "slateblue",
    "slategray",
    "slategrey",
    "springgreen",
    "steelblue",
    "tan",
    "teal",
    "thistle",
    "tomato",
    "turquoise",
    "violet",
    "wheat",
    "yellowgreen",
]
# random.shuffle(plot_colors)
random.Random(4).shuffle(plot_colors)
# plot_colors

In [ ]:
# Dictionaries for plots
var_colors_dict = {
    "hru_actet": "red",
    "recharge": "brown",
    "net_rain": "blue",
    "net_snow": "lightblue",
    "net_ppt": "darkblue",
    "sroff": "lightgreen",
    "ssres_flow": "green",
    "ssres_flow_vol": "green",
    "gwres_flow": "chocolate",
    "gwres_sink": "black",
    "snowmelt": "mediumpurple",
    "gwres_stor": "darkgreen",
    "gwres_stor_change": "darkgreen",
    "ssres_stor": "green",
    "unused_potet": "orange",
}

# 'legendonly'
leg_only_dict = {
    "hru_actet": "legendonly",
    "recharge": "legendonly",
    "net_rain": "legendonly",
    "net_snow": "legendonly",
    "net_ppt": True,
    "sroff": True,
    "ssres_flow": "legendonly",
    "ssres_flow_vol": "legendonly",
    "gwres_flow": True,
    "gwres_sink": "legendonly",
    "snowmelt": "legendonly",
    "gwres_stor": "legendonly",
    "gwres_stor_change": "legendonly",
    "ssres_stor": "legendonly",
    "unused_potet": "legendonly",
}

In [ ]:
if poi_id_sel:
    hru_list = hru_poi_dict[
        poi_id_sel
    ]  # returns a list of all upstream contributing hrus

    fig = plotly.subplots.make_subplots(
        rows=3,
        cols=1,
        shared_xaxes="columns",
        # shared_yaxes = 'columns',
        start_cell="top-left",
        vertical_spacing=0.1,
        y_title=f"{output_var_sel}, {da.units}",
        subplot_titles=[
            "Annual mean",
            "Monthly mean",
            "Daily",
        ],
        specs=[
            [{"type": "scatter"}],
            [{"type": "scatter"}],
            [{"type": "scatter"}],
        ],
    )
    fig.update_layout(
        title_text=f'The NHM simulated HRU {output_var_sel} for basin <br> {poi_id_sel}, {poi_df.loc[poi_df.poi_id == poi_id_sel, "poi_name"].values[0]}',  #
        width=900,
        height=700,
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="right", x=10.0),
        # legend_tracegroupgap = 5,
        font=dict(family="Arial", size=14, color="#7f7f7f"),  # font color
        paper_bgcolor="linen",
        plot_bgcolor="white",
    )

    fig.update_layout(
        title_automargin=True,
        title_font_color="black",
        title_font_size=20,
        title_x=0.5,
        title_y=0.945,
        title_xref="container",
        title_xanchor="center",
    )

    # fig.update_xaxes(range = [daily_plot_df.index[0], daily_plot_df.index[-1]])

    fig.update_layout(font_color="black")
    fig.update_layout(legend={"title": "Area"})

    fig.update_xaxes(ticks="inside", tickwidth=2, tickcolor="black", ticklen=10)
    fig.update_yaxes(ticks="inside", tickwidth=2, tickcolor="black", ticklen=10)

    fig.update_xaxes(
        showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
    )
    fig.update_yaxes(
        showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
    )

    fig.update_traces(hovertemplate=None)
    fig.update_layout(hovermode="x unified")
    fig.update_layout(
        hoverlabel=dict(
            bgcolor="linen",
            font_size=13,
            font_family="Rockwell",
        )
    )

    ######################################################

    annual_fig = go.Figure()

    # subset to selcted gage one gage
    df_basin = df_output_var_annual_basin.loc[
        df_output_var_annual_basin["nhm_id"].isin(hru_poi_dict[poi_id_sel])
    ]
    df_basin.set_index(
        ["time", "nhm_id"], inplace=True, drop=True
    )  # resets the index to that new value and type

    # Calculate basin recharge from individual HRU contributions for plotting
    df_basin_plot1 = df_basin.groupby(level="time").sum()
    df_basin_plot1["output_var"] = df_basin_plot1["vol"] / df_basin_plot1["hru_area"]

    annual_fig.add_trace(
        go.Scatter(
            x=df_basin_plot1.index,
            y=(df_basin_plot1.output_var).ravel().tolist(),
            mode="lines",
            name=f"Drainage area",
            showlegend=True,
            legendgroup="poi_basin",
            # marker=dict(color='lightblue'),
            line=dict(
                color="lightblue",
                width=5,
                # dash='dot'
            ),
        )
    )

    for (
        value
    ) in (
        hru_list
    ):  # I fixed this below to read right from the xarray, need to fix the hru_list as well.
        hru_id_sel = value
        color_sel = hru_list.index(hru_id_sel)
        ds_sub = (
            output_var_annual.where((output_var_annual.nhm_id == hru_id_sel), drop=True)
        ).sel(
            time=slice(plot_start_date, plot_end_date)
        )  # have to fix to subset to new date range
        annual_fig.add_trace(
            go.Scatter(
                x=ds_sub.time,
                y=(ds_sub.values).ravel().tolist(),
                mode="lines",
                name=f"HRU {hru_id_sel}",
                showlegend=True,
                visible="legendonly",
                legendgroup=hru_id_sel,
                # marker=dict(color=plot_colors[color_sel]),
                line=dict(
                    color=plot_colors[color_sel],
                    width=2,
                    # dash='dot'
                ),
            )
        )

    monthly_fig = go.Figure()

    # subset to selcted gage one gage
    df_basin = df_output_var_monthly_basin.loc[
        df_output_var_monthly_basin["nhm_id"].isin(hru_poi_dict[poi_id_sel])
    ]
    df_basin.set_index(
        ["time", "nhm_id"], inplace=True, drop=True
    )  # resets the index to that new value and type

    # Calculate basin recharge from individual HRU contributions for plotting
    df_basin_plot1 = df_basin.groupby(level="time").sum()
    df_basin_plot1["output_var"] = df_basin_plot1["vol"] / df_basin_plot1["hru_area"]

    monthly_fig.add_trace(
        go.Scatter(
            x=df_basin_plot1.index,
            y=(df_basin_plot1.output_var).ravel().tolist(),
            mode="lines",
            name=poi_id_sel,
            showlegend=False,
            legendgroup="poi_basin",
            # marker=dict(color='lightblue'),
            line=dict(
                color="lightblue",
                width=5,
                # dash='dot'
            ),
        )
    )

    for (
        value
    ) in (
        hru_list
    ):  # I fixed this below to read right from the xarray, need to fix the hru_list as well.
        hru_id_sel = value
        color_sel = hru_list.index(hru_id_sel)
        ds_sub = (
            output_var_monthly.where(
                (output_var_monthly.nhm_id == hru_id_sel), drop=True
            )
        ).sel(time=slice(plot_start_date, plot_end_date))
        monthly_fig.add_trace(
            go.Scatter(
                x=ds_sub.time,
                y=(ds_sub.values).ravel().tolist(),
                mode="lines",
                name=hru_id_sel,
                showlegend=False,
                visible="legendonly",
                legendgroup=hru_id_sel,
                # marker=dict(color=plot_colors[color_sel]),
                line=dict(
                    color=plot_colors[color_sel],
                    width=2,
                    # dash='dot'
                ),
            )
        )
    daily_fig = go.Figure()

    # subset to selcted gage one gage
    df_basin = df_output_var_daily_basin.loc[
        df_output_var_daily_basin["nhm_id"].isin(hru_poi_dict[poi_id_sel])
    ]
    df_basin.set_index(
        ["time", "nhm_id"], inplace=True, drop=True
    )  # resets the index to that new value and type

    # Calculate basin recharge from individual HRU contributions for plotting
    df_basin_plot1 = df_basin.groupby(level="time").sum()
    df_basin_plot1["output_var"] = df_basin_plot1["vol"] / df_basin_plot1["hru_area"]

    daily_fig.add_trace(
        go.Scatter(
            x=df_basin_plot1.index,
            y=(df_basin_plot1.output_var).ravel().tolist(),
            mode="lines",
            name=poi_id_sel,
            showlegend=False,
            legendgroup="poi_basin",
            # marker=dict(color='lightblue'),
            line=dict(
                color="lightblue",
                width=5,
                # dash='dot'
            ),
        )
    )

    for (
        value
    ) in (
        hru_list
    ):  # I fixed this below to read right from the xarray, need to fix the hru_list as well.
        hru_id_sel = value
        color_sel = hru_list.index(hru_id_sel)
        ds_sub = (
            output_var_daily.where((output_var_daily.nhm_id == hru_id_sel), drop=True)
        ).sel(time=slice(plot_start_date, plot_end_date))
        daily_fig.add_trace(
            go.Scatter(
                x=ds_sub.time,
                y=(ds_sub.values).ravel().tolist(),
                mode="lines",
                name=hru_id_sel,
                showlegend=False,
                visible="legendonly",
                legendgroup=hru_id_sel,
                # marker=dict(color=plot_colors[color_sel]),
                line=dict(
                    color=plot_colors[color_sel],
                    width=2,
                    # dash='dot'
                ),
            )
        )

    for t in annual_fig.data:
        fig.append_trace(t, row=1, col=1)

    for t in monthly_fig.data:
        fig.append_trace(t, row=2, col=1)

    for t in daily_fig.data:
        fig.append_trace(t, row=3, col=1)

    fig.show()

In [ ]:
if poi_id_sel:
    fig = plotly.subplots.make_subplots(
        rows=3,
        cols=1,
        shared_xaxes="columns",
        # shared_yaxes = 'columns',
        start_cell="top-left",
        vertical_spacing=0.1,
        y_title="Water flux, cubic-feet per second",
        subplot_titles=[
            "Annual mean",
            "Monthly mean",
            "Daily",
        ],
        specs=[
            [{"type": "scatter"}],
            [{"type": "scatter"}],
            [{"type": "scatter"}],
        ],
    )
    fig.update_layout(
        title_text=f'The NHM basin water budget flux rates for <br> {poi_id_sel}, {poi_df.loc[poi_df.poi_id == poi_id_sel, "poi_name"].values[0]}',  #
        width=900,
        height=700,
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="right", x=10.0),
        # legend_tracegroupgap = 5,
        font=dict(family="Arial", size=14, color="#7f7f7f"),  # font color
        paper_bgcolor="linen",
        plot_bgcolor="white",
    )

    fig.update_layout(
        title_automargin=True,
        title_font_color="black",
        title_font_size=20,
        title_x=0.5,
        title_y=0.945,
        title_xref="container",
        title_xanchor="center",
    )

    # fig.update_xaxes(range = [daily_plot_df.index[0], daily_plot_df.index[-1]])

    fig.update_layout(font_color="black")
    fig.update_layout(
        legend={"title": "NHM output variable"}
    )  # <--- add only this line

    fig.update_xaxes(ticks="inside", tickwidth=2, tickcolor="black", ticklen=10)
    fig.update_yaxes(ticks="inside", tickwidth=2, tickcolor="black", ticklen=10)

    fig.update_xaxes(
        showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
    )
    fig.update_yaxes(
        showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
    )

    fig.update_traces(hovertemplate=None)

    fig.update_layout(hovermode="x unified")
    fig.update_layout(
        hoverlabel=dict(
            bgcolor="linen",
            font_size=13,
            font_family="Rockwell",
        )
    )

    daily_fig = go.Figure()
    annual_fig = go.Figure()
    monthly_fig = go.Figure()
    ######################################################
    for var in output_var_list:
        color_sel = var_colors_dict[var]
        leg_only = leg_only_dict[var]

        # JLM TODO: remove this block
        with xr.open_dataset(custom_output_file) as model_output:
            output_var1 = model_output.sel(time=slice(plot_start_date, plot_end_date))
            ##################################################
            output_var_daily1 = getattr(output_var1, var)
            output_var_annual1 = (
                getattr(output_var1, var).resample(time="A-SEP").mean()
            )  # Water year
            output_var_monthly1 = (
                getattr(output_var1, var).resample(time="m").mean()
            )  # monthly

        # JLM TODO: keep this block
        with xr.load_dataarray(out_dir / f"{var}.nc") as da:
            # these machinations are to keep downstream things as they were before some refactoring
            da = da.to_dataset().rename_dims({"nhm_id": "nhru"})[da.name]
            output_var_daily2 = da.sel(time=slice(plot_start_date, plot_end_date))
            output_var_monthly2 = output_var_daily2.resample(time="m").mean()
            # Water year annual
            output_var_annual2 = output_var_daily2.resample(time="A-SEP").mean()

        # JLM TODO: remove this block
        xr.testing.assert_equal(output_var_daily1, output_var_daily2)
        xr.testing.assert_equal(output_var_monthly1, output_var_monthly2)
        xr.testing.assert_equal(output_var_annual1, output_var_annual2)

        # JLM TODO: Remove this. This is here to demonstrated that the above runs fine 
        # while running after this causes a kernel restart. This happens even with the 
        # custom_output_file in-place.
        adsf
        
        # Create a dataframe of MONTHLY recharge values for each HRU
        ds_output_var_monthly_basin1 = output_var_monthly1.copy()
        df_output_var_monthly_basin1 = ds_output_var_monthly_basin1.to_dataframe(
            dim_order=["time", "nhru"]
        )
        df_output_var_monthly_basin1.reset_index(inplace=True, drop=False)
        df_output_var_monthly_basin1.rename(columns={var: "output_var"}, inplace=True)

        # add the HRU area to the dataframe
        df_output_var_monthly_basin1 = df_output_var_monthly_basin1.merge(
            hru_area_df, how="left", right_index=True, left_on="nhm_id"
        )

        # Add output_var volume to the dataframe
        df_output_var_monthly_basin1["vol"] = (
            df_output_var_monthly_basin1["output_var"]
            * (df_output_var_monthly_basin1["hru_area"] * 6272640)
        ) / (12 * 12 * 12)
        df_output_var_monthly_basin1["cfs"] = (
            (
                df_output_var_monthly_basin1["output_var"]
                * (df_output_var_monthly_basin1["hru_area"] * 6272640)
            )
            / (12 * 12 * 12)
        ) / 86400

        # Drop unneeded columns
        df_output_var_monthly_basin1.drop(columns=["nhru"], inplace=True)

        # Create a dataframe of Daily recharge values for each HRU
        ds_output_var_daily_basin1 = output_var_daily1.copy()
        df_output_var_daily_basin1 = ds_output_var_daily_basin1.to_dataframe(
            dim_order=["time", "nhru"]
        )
        df_output_var_daily_basin1.reset_index(inplace=True, drop=False)
        df_output_var_daily_basin1.rename(columns={var: "output_var"}, inplace=True)

        # add the HRU area to the dataframe
        df_output_var_daily_basin1 = df_output_var_daily_basin1.merge(
            hru_area_df, how="left", right_index=True, left_on="nhm_id"
        )

        # Add recharge volume to the dataframe
        df_output_var_daily_basin1["vol"] = (
            df_output_var_daily_basin1["output_var"]
            * (df_output_var_daily_basin1["hru_area"] * 6272640)
        ) / (12 * 12 * 12)
        df_output_var_daily_basin1["cfs"] = (
            (
                df_output_var_daily_basin1["output_var"]
                * (df_output_var_daily_basin1["hru_area"] * 6272640)
            )
            / (12 * 12 * 12)
        ) / 86400

        # Drop unneeded columns
        df_output_var_daily_basin1.drop(columns=["nhru"], inplace=True)

        # Create a dataframe of ANNUAL recharge values for each HRU
        ds_output_var_annual_basin1 = output_var_annual1.copy()
        df_output_var_annual_basin1 = ds_output_var_annual_basin1.to_dataframe(
            dim_order=["time", "nhru"]
        )
        df_output_var_annual_basin1.reset_index(inplace=True, drop=False)
        df_output_var_annual_basin1.rename(columns={var: "output_var"}, inplace=True)

        df_output_var_annual_basin1 = df_output_var_annual_basin1.merge(
            hru_area_df, how="left", right_index=True, left_on="nhm_id"
        )

        # Add recharge volume to the dataframe
        df_output_var_annual_basin1["vol"] = (
            df_output_var_annual_basin1["output_var"]
            * (df_output_var_annual_basin1["hru_area"] * 6272640)
        ) / (
            12 * 12 * 12
        )  # Cubic-feet
        df_output_var_annual_basin1["cfs"] = (
            (
                df_output_var_annual_basin1["output_var"]
                * (df_output_var_annual_basin1["hru_area"] * 6272640)
            )
            / (12 * 12 * 12)
        ) / 86400  # Cubic-feet/sec

        # Drop unneeded columns
        df_output_var_annual_basin1.drop(columns=["nhru"], inplace=True)

        ###################################################################################################################

        # subset to selcted gage one gage
        df_basin1 = df_output_var_annual_basin1.loc[
            df_output_var_annual_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
        ]
        df_basin1.set_index(
            ["time", "nhm_id"], inplace=True, drop=True
        )  # resets the index to that new value and type

        # Calculate basin recharge from individual HRU contributions for plotting
        df_basin_plot1 = df_basin1.groupby(level="time").sum()
        df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
            df_basin_plot1["hru_area"] * 6272640
        )  # back to inches
        # df_basin_plot['cfs'] =

        annual_fig.add_trace(
            go.Scatter(
                x=df_basin_plot1.index,
                y=(df_basin_plot1.cfs).ravel().tolist(),
                # x=year_list,
                # y= (gdf.loc[gdf.nhm_id == hru_id_sel, year_list].values).ravel().tolist(),
                mode="lines",
                name=var,
                visible=leg_only,
                showlegend=True,
                legendgroup=var,
                # marker=dict(color='lightblue'),
                line_shape="vh",
                line=dict(
                    color=color_sel,
                    width=2,
                    # dash='dot'
                ),
            )
        )

        # subset to selcted gage one gage
        df_basin1 = df_output_var_monthly_basin1.loc[
            df_output_var_monthly_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
        ]
        df_basin1.set_index(
            ["time", "nhm_id"], inplace=True, drop=True
        )  # resets the index to that new value and type

        # Calculate basin recharge from individual HRU contributions for plotting
        df_basin_plot1 = df_basin1.groupby(level="time").sum()
        df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
            df_basin_plot1["hru_area"] * 6272640
        )  # back to inches

        monthly_fig.add_trace(
            go.Scatter(
                x=df_basin_plot1.index,
                y=(df_basin_plot1.cfs).ravel().tolist(),
                # x=year_list,
                # y= (gdf.loc[gdf.nhm_id == hru_id_sel, year_list].values).ravel().tolist(),
                mode="lines",
                name=var,
                visible=leg_only,
                showlegend=False,
                legendgroup=var,
                # marker=dict(color='lightblue'),
                line_shape="vh",
                line=dict(
                    color=color_sel,
                    width=2,
                    # dash='dot'
                ),
            )
        )

        # subset to selcted gage one gage
        df_basin1 = df_output_var_daily_basin1.loc[
            df_output_var_daily_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
        ]
        df_basin1.set_index(
            ["time", "nhm_id"], inplace=True, drop=True
        )  # resets the index to that new value and type

        # Calculate basin recharge from individual HRU contributions for plotting
        df_basin_plot1 = df_basin1.groupby(level="time").sum()
        df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
            df_basin_plot1["hru_area"] * 6272640
        )  # back to inches

        daily_fig.add_trace(
            go.Scatter(
                x=df_basin_plot1.index,
                y=(df_basin_plot1.cfs).ravel().tolist(),
                # x=year_list,
                # y= (gdf.loc[gdf.nhm_id == hru_id_sel, year_list].values).ravel().tolist(),
                mode="lines",
                name=var,
                visible=leg_only,
                showlegend=False,
                legendgroup=var,
                # marker=dict(color='lightblue'),
                line_shape="vh",
                line=dict(
                    color=color_sel,
                    width=2,
                    # dash='dot'
                ),
            )
        )

    for t in annual_fig.data:
        fig.append_trace(t, row=1, col=1)

    for t in monthly_fig.data:
        fig.append_trace(t, row=2, col=1)

    for t in daily_fig.data:
        fig.append_trace(t, row=3, col=1)

    fig.show()